# Translated Periodicity using a simple Block
The mesh shown below has periodic faces on the z and y axis. The code below shows how we find the translated periodicity in the y and z directions.

Steps Overview
1. Find Connectivity - This splits the faces of all the blocks so that they match 
2. Build the Block connectivity matrix using connectivity - Uses the split faces to determine which blocks are connected. value of 1 = connected, 0 = not searched yet, -1 = not connected 
3. Search for connected faces on the z and y axis - Uses the block connectivity to find the outer connected faces. So, all the top faces for instance. 
4. Find translated periodicity - Uses the connected faces e.g. the left and right, or top and bottom, or front and back. Checks the connected faces to see if any of them match faces from the other side. 

![image.png](tutorial_images/cmc009-overview.png)

In [ ]:
# Install Plot3D
!pip install plot3d

In [14]:
# Import functions
import os, pickle
from plot3d import read_plot3D, connectivity_fast,translational_periodicity, write_plot3D, Direction, split_blocks, block_connection_matrix, find_bounding_faces
from plot3d import outer_face_dict_to_list, match_faces_dict_to_list
import numpy as np
#%% Find connectivity 
def dump_data(data):
    with open('block_data.pickle','wb') as f:
        pickle.dump(data,f)

def read_data():
    with open('block_data.pickle','rb') as f:
        return pickle.load(f)


In [15]:
# Load the mesh
blocks = read_plot3D('iso65_64blocks.p3d',False)
write_plot3D('iso65_64blocks.xyz',blocks,True)
n = len(blocks)
print(f'Number of blocks: {n}')

Number of blocks: 64


## Step 1: Find Connectivity
The code below finds the connectivity and creates the pickle file storing the connectivity 

In [16]:
print('Finding connectivity')
face_matches, outer_faces = connectivity_fast(blocks)
[m.pop('match',None) for m in face_matches] # Remove the dataframe to save as pickle file 
print('Organizing split and outerfaces')
all_faces = match_faces_dict_to_list(blocks,face_matches)
all_faces.extend(outer_face_dict_to_list(blocks,outer_faces))
all_faces = [m.to_dict() for m in all_faces]
data = {
            "face_matches":face_matches, 
            "outer_faces":outer_faces,
            "all_faces":all_faces
        }
dump_data(data)

Finding connectivity
gcd to use 16


Checking connections block 62 with 63: 100%|██████████| 363/363 [00:18<00:00, 19.73it/s]


Organizing split and outerfaces


## Step 2: Build Block to Block connectivity matrix
Block to block connectivity makes it easier to search for which blocks are connected. For example `print(c[10,4])` if this is 1 then that means blocks 10 and 4 are touching. 

Figure below shows an example of the connections for block 704. 

![block connectivity](tutorial_images/cmc009-block-connectivity.png)

In [19]:
data = read_data()
print('Creating block connection matrix')
c = block_connection_matrix(blocks,all_faces)
data["connectivity_matrix"]=c
dump_data(data)


Creating block connection matrix


Building block to block connectivity matrix: checking 62: 100%|██████████| 2016/2016 [00:02<00:00, 999.50it/s] 


These are the blocks where block 2 is connected to. We have blocks 0,3,6, and 16. Block 2 is automtically connected with itself. 

In [20]:
print("connectivity matrix shape: " + str(c.shape[0]) + ' by ' + str(c.shape[1]))
np.where(c[2,:]==1)

connectivity matrix shape: 64 by 64


(array([ 0,  2,  3,  6, 16], dtype=int64),)

## Step 3: Find bounding faces
These are faces on either the left or right side of the block.

In [21]:
data = read_data()    
all_faces = data['all_faces']
connectivity_matrix = data['connectivity_matrix']

#%% Find bounding Faces
lower_bound, upper_bound,_,_ = find_bounding_faces(blocks,connectivity_matrix,all_faces,"z")
left_bound, right_bound,_,_ = find_bounding_faces(blocks,connectivity_matrix,all_faces,"y")
data['lower_bound'] = lower_bound
data['upper_bound'] = upper_bound
data['left_bound'] = left_bound
data['right_bound'] = right_bound
dump_data(data)

## Step 4: Find Periodicity
In this step we determine the connected faces in the y-direction as well as the z-direction. The program will automatically inform the user if it's not periodic due to not all faces matching. If this is the case then the faces should be plotted using paraview and the user needs to check with their grid generator or manually add in the connectivity. 

In [24]:
#%% Use bounding faces to find periodicity
data = read_data()
lower_bound = data['lower_bound']; upper_bound = data['upper_bound']
left_bound = data['left_bound']; right_bound = data['right_bound']
y_periodic_faces_export, periodic_faces = translational_periodicity(blocks,left_bound,right_bound,translational_direction='y')
z_periodic_faces_export, periodic_faces = translational_periodicity(blocks,lower_bound,upper_bound,translational_direction='z')
data['z_periodic'] = z_periodic_faces_export
data['y_periodic'] = y_periodic_faces_export
dump_data(data)

#%%  Lets check with faces are not periodic in the y-direction
data = read_data()
y_periodic_faces_export = data['y_periodic']
left_bound = data['left_bound']; right_bound = data['right_bound']

left_periodic_blocks_found = [p['block1']['block_index'] for p in y_periodic_faces_export]
left_faces_missing = [l for l in left_bound if l['block_index'] not in left_periodic_blocks_found]
print('Left faces missing')
[print(l) for l in left_faces_missing]

right_periodic_blocks_found = [p['block2']['block_index'] for p in y_periodic_faces_export]
right_faces_missing = [r for r in right_bound if r['block_index'] not in right_periodic_blocks_found]
print('Right faces missing')
[print(r) for r in right_faces_missing]

Checking connections block 17 with 54:   0%|          | 0/16 [00:00<?, ?it/s]



Not periodic y


Checking connections block 45 with 63: 100%|██████████| 16/16 [00:00<00:00, 43.13it/s]



Periodic z
Left faces missing
{'IMIN': 0, 'JMIN': 0, 'KMIN': 0, 'IMAX': 0, 'JMAX': 16, 'KMAX': 16, 'id': 0, 'block_index': 3}
{'IMIN': 0, 'JMIN': 0, 'KMIN': 0, 'IMAX': 0, 'JMAX': 16, 'KMAX': 16, 'id': 0, 'block_index': 1}
{'IMIN': 0, 'JMIN': 0, 'KMIN': 0, 'IMAX': 0, 'JMAX': 16, 'KMAX': 16, 'id': 0, 'block_index': 2}
{'IMIN': 0, 'JMIN': 0, 'KMIN': 0, 'IMAX': 0, 'JMAX': 16, 'KMAX': 16, 'id': 0, 'block_index': 17}
{'IMIN': 0, 'JMIN': 0, 'KMIN': 0, 'IMAX': 0, 'JMAX': 16, 'KMAX': 16, 'id': 0, 'block_index': 10}
{'IMIN': 0, 'JMIN': 0, 'KMIN': 0, 'IMAX': 0, 'JMAX': 16, 'KMAX': 16, 'id': 0, 'block_index': 16}
{'IMIN': 0, 'JMIN': 0, 'KMIN': 0, 'IMAX': 0, 'JMAX': 16, 'KMAX': 16, 'id': 0, 'block_index': 19}
{'IMIN': 0, 'JMIN': 0, 'KMIN': 0, 'IMAX': 0, 'JMAX': 16, 'KMAX': 16, 'id': 0, 'block_index': 24}
{'IMIN': 0, 'JMIN': 0, 'KMIN': 0, 'IMAX': 0, 'JMAX': 16, 'KMAX': 16, 'id': 0, 'block_index': 8}
{'IMIN': 0, 'JMIN': 0, 'KMIN': 0, 'IMAX': 0, 'JMAX': 16, 'KMAX': 16, 'id': 0, 'block_index': 11}
{'I

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]